## 0. Install and Import Dependencies

In [1]:
# Install pytorch
#https://pytorch.org/get-started/locally/
# Install easyocr 
!pip install easyocr

DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Read in images or video

In [23]:
#IMAGE_PATH = 'nimek.webp'
IMAGE_PATH = 'sign.jpg'

In [24]:
reader = easyocr.Reader(['en'])
result = reader.readtext(IMAGE_PATH)
full=''
for i in result:
    full+=" "
    full+=i[1]
full

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


' Navaner 0 Tut 0s  8 ! Crocin udEMAabd Chee Wtui Konl Porroe 0u udom- I2cul 90 157 0105 1 8 Vor nole nfuenibtiod pe Dceduct; Vioit Wiy @ Ucincani  Wectia PARACETAMOL FAST Imoe RELEASETTABLETS 1 8 Kocle; 2p Mry5  Crocin Adatg eml OM 0 8 eeem ! { Crocin S Metuiuia runtc Advine ULI Wucial E I Piq rtmir 6 TutIva IowmbtlymlIcecl Matukorun Jg cep dotnuic nn Unear 2 End 6 1 u KUp5T rocin Ourmcdl" 12C7 ci 2015 9a'

In [39]:
import pandas as pd
from rapidfuzz import process, fuzz
import re

# Step 1: Clean and normalize dataset entries
def clean_text(text):
    if pd.isna(text):
        return ""
    # Remove non-alphanumeric characters except spaces, convert to lowercase
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower()).strip()

def normalize_text(text):
    # Sort words alphabetically after cleaning
    return " ".join(sorted(clean_text(text).split()))

# Step 2: Preprocess dataset names for better matching
def preprocess_dataset(dataset_names):
    return [normalize_text(name) for name in dataset_names]

# Step 3: Perform word-level fuzzy matching
def find_word_level_matches(input_name, dataset_names, normalized_dataset, limit=5):
    # Normalize the input only for matching purposes
    normalized_input = normalize_text(input_name)
    
    # Use rapidfuzz to find the best matches with normalized dataset
    matches = process.extract(normalized_input, normalized_dataset, scorer=fuzz.token_set_ratio, limit=limit)
    
    # Retrieve original names using match indices
    results = [(dataset_names[idx], score) for _, score, idx in matches]
    return results

# Step 4: Load data and perform matching
if __name__ == "__main__":
    # Load the dataset
    file_path = "generic.csv"  # Replace with your file path
    try:
        data = pd.read_csv(file_path)
        if 'DRUG NAME' not in data.columns:
            raise ValueError("The CSV file must contain a 'DRUG NAME' column.")
        
        # Original and normalized dataset
        dataset_names = data['DRUG NAME'].tolist()  # Original names
        normalized_dataset = preprocess_dataset(dataset_names)  # Preprocessed for matching

        # Example input (unchanged noisy input)
        input_name = full

        # Find matches
        matches = find_word_level_matches(input_name, dataset_names, normalized_dataset)
        
        for original_name, score in matches:
            if not data.loc[data['DRUG NAME'] == original_name, 'DRUG PRICE'].empty:
                price = data.loc[data['DRUG NAME'] == original_name, 'DRUG PRICE'].iloc[0]
            else:
                price = None  # or handle the case appropriately

    
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: 'DRUG PRICE'


In [41]:
file_path = "generic.csv"  # Replace with your file path
data = pd.read_csv(file_path)
data.columns

Index(['DRUG NAME', 'DRUG PRICE '], dtype='object')